In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from rdkit import Chem

In [ ]:
molecules = Chem.SDMolSupplier("../../data/RECETOX_GC-EI-MS_20201028.sdf")
top5 = pd.read_csv("../data/matchms_top5_comparison.tsv", sep='\t')

In [ ]:
def is_spectrum_for_compound(compund_name, spectrum_name):
    options = [compund_name + x for x in ["", "_isomer1", "_isomer2", " isomer 1", " isomer 2"]]
    return spectrum_name in options

In [ ]:
def get_matching_rows(df, query_name, reference_name):
    return df[df.apply(lambda x: is_spectrum_for_compound(x[query_name], x[reference_name]), axis=1)]

In [ ]:
def has_halogen_atoms(mol):
    # Check if the molecule contains any halogen atoms
    for atom in mol.GetAtoms():
        if atom.GetSymbol() in ['F', 'Cl', 'Br', 'I']:
            return True

    return False

In [ ]:
def has_atom(mol, atom):
    for mol_atom in mol.GetAtoms():
        if mol_atom.GetSymbol() == atom:
            return True

    return False

In [ ]:
# Define a function to map the true columns to a list of names
def get_true_names(row, df):
    return [col for col in df.columns[11:17] if row[col]]

In [ ]:
def append_classes(df,molecules,left_on):
    class_names = pd.DataFrame({
        "class" : [m.GetProp("Class") for m in molecules],
        "superclass" : [m.GetProp("Superclass") for m in molecules],
        "molname" : [m.GetProp("NAME") for m in molecules],
        "n_atoms" : [m.GetNumAtoms() for m in molecules],
        "n_bonds" : [m.GetNumBonds() for m in molecules],
        "has_halogen": [has_halogen_atoms(m) for m in molecules],
        "smiles" : [m.GetProp("SMILES") for m in molecules],
        "Cl": [has_atom(m, 'Cl') for m in molecules],
        "Br": [has_atom(m, 'Br') for m in molecules],
        "F": [has_atom(m, 'F') for m in molecules],
        "S": [has_atom(m, 'S') for m in molecules],
        "P": [has_atom(m, 'P') for m in molecules],
        "Si": [has_atom(m, 'Si') for m in molecules],
    })
    merged_df = pd.merge(df, class_names, left_on=left_on, right_on='molname')
    return merged_df

In [ ]:
top5_same = get_matching_rows(top5, 'query', 'reference')
merged_top5_same = append_classes(top5_same,molecules, 'query')
# Apply the function to each row of the DataFrame
merged_top5_same['true_names'] = merged_top5_same.apply(lambda row: get_true_names(row, merged_top5_same), axis=1)
# Convert the list of names to a comma-separated string
merged_top5_same['true_names'] = merged_top5_same['true_names'].apply(lambda lst: ', '.join(lst))

In [ ]:
sns.set_style(style='white')
plt.figure(figsize=(17, 5))

# Concatenate the DataFrames in df1_list and add a 'value' column with the value 'matches'.
df1 = merged_top5_same[['query', 'reference', 'true_names', 'CosineHungarian_0.01_0.0_1.0_matches']].copy()

# Concatenate the DataFrames in df2_list and add a 'value' column with the value 'scores'.
df2 = merged_top5_same[['query', 'reference', 'true_names', 'CosineHungarian_0.01_0.0_1.0_scores']].copy()

# Concatenate df1 and df2 into a single DataFrame.
df_cat = pd.concat([df1, df2])
mdf = pd.melt(df_cat, id_vars=['query', 'reference', 'true_names'], var_name=['Number'])      # MELT

# Set the color palette
colors = ['yellow', 'deepskyblue']
sns.set_palette(sns.color_palette(colors))

ax = sns.boxplot(x="true_names", y="value", hue="Number", 
                data=mdf, hue_order=['CosineHungarian_0.01_0.0_1.0_matches',np.nan],
                medianprops={'color': 'darkgreen', 'linewidth': 4.0},
                flierprops={'marker': 'o', 'markersize': 10, 'markerfacecolor': 'none'})  # RUN PLOT   
ax2 = ax.twinx()

sns.boxplot(ax=ax2,x='true_names', y='value', hue='Number',
            data=mdf, hue_order=[np.nan, 'CosineHungarian_0.01_0.0_1.0_scores'], 
            medianprops={'color': 'b', 'linewidth': 4.0}, 
            flierprops={'marker': 'o', 'markersize': 10, 'markerfacecolor': 'none'})

ax.legend_.remove()
ax.set_ylim([0, 6])  # Set y-axis limits
ax.yaxis.set_major_locator(plt.MultipleLocator(1))  # Set major tick marks
ax.set_ylabel('Match values')  # Set y-axis label
ax.yaxis.label.set_size(20)  # Set font size of y-axis label
ax.set_xlabel('Chemical composition', fontsize=20)  # Set x-axis label and font size
ax.tick_params(axis='x', labelsize=13)  # Set font size of x-axis tick labels
ax.tick_params(axis='y', labelsize=13)  # Set font size of y-axis tick labels
ax.yaxis.labelpad = 10
ax.xaxis.labelpad = 10

# Add x-axis labels with group sizes
grouped_df = merged_top5_same.groupby('true_names')['query'].apply(list).reset_index(name='queries')
xticklabels = [f"{group} ({len(x)})" for group, x in grouped_df.values]
ax.set_xticklabels(xticklabels, rotation=90, ha='right')

ax2.legend_.remove()
ax2.set_ylim([0, 1.2])  # Set y-axis limits
ax2.yaxis.set_major_locator(plt.MultipleLocator(0.2))  # Set major tick marks
ax2.set_ylabel('Score values')  # Set y-axis label
ax2.yaxis.label.set_size(20)  # Set font size of y-axis label
ax2.tick_params(axis='y', labelsize=12)  # Set font size of y-axis tick labels
ax2.yaxis.labelpad = 10

# Change the legend labels
handles, labels = ax.get_legend_handles_labels()
labels[0] = 'Matches'
labels[1] = 'Scores'
ax.legend(handles, labels, loc='upper right', fontsize=14)

plt.savefig('../Python_scripts/chemical_composition_boxplot.png', dpi=300, bbox_inches='tight')  # SAVE PLOT AS PNG FILE
plt.show()
plt.clf()
plt.close()